# Known-Artist Live Song ID: A Hashprint Approach

This notebook is a python implementation of the [original matlab version] (http://pages.hmc.edu/ttsai/assets/livesongid.tar.gz). The pipeline consists of the following modules:

1. Preprocessing with Constant-Q Transform (CQT)
2. Learning PCA filters
3. Obtaining hashprint representation by applying PCA filters to preprocessed CQT matrix
4. Generating database with the hashprint of pitch-shifted CQT matrix of reference tracks
5. Matching query against database

In [1]:
GIT_DIR = '/home/mjenrungrot/workspace/live-song-id/'
%cd $GIT_DIR

/home/mjenrungrot/fast.ai/live-song-id


## Data Preprocessing

For a given artist, we first preprocess the list of reference tracks by taking CQT transform for each audio file.

In [2]:
from preprocess import *
import os
from utils import *

In [3]:
# audio_path - where audio files are stored
# list_dir - where list of softlinks are stored
# cqt_dir - where results will be stored
audio_path = '/home/mjenrungrot/SoftLinks/' # change this
list_dir = audio_path + '/Lists/'
cqt_dir = '/home/mjenrungrot/ttmp/livesong_results/' # change this
artist = 'taylorswift'
out_dir = os.path.join(cqt_dir, artist+'_out')
file_paths = get_allpaths(artist, list_dir)

In [4]:
%cd $cqt_dir
%mkdir $out_dir
%cd $out_dir

f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'w')
for cur_file in file_paths:
    print('==> Computing CQT of %s'%cur_file)
    y, sr = librosa.load(audio_path + cur_file + '.wav')
    Q = librosa.cqt(y, sr=sr, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)
    logQ = preprocess(Q, 3)
    cur_file_path = os.path.join(os.getcwd(), cur_file + '.dat')
    np.savetxt(cur_file_path, logQ)
    f.write(cur_file_path + '\n')
f.close()


/data1/mjenrungrot/livesong_results
mkdir: cannot create directory ‘/home/mjenrungrot/ttmp/livesong_results/taylorswift_out’: File exists
/data1/mjenrungrot/livesong_results/taylorswift_out
==> Computing CQT of taylorswift_ref1


/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):
/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):


==> Computing CQT of taylorswift_ref2


/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):
/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):


==> Computing CQT of taylorswift_ref3
==> Computing CQT of taylorswift_ref4
==> Computing CQT of taylorswift_ref5
==> Computing CQT of taylorswift_ref6
==> Computing CQT of taylorswift_ref7
==> Computing CQT of taylorswift_ref8
==> Computing CQT of taylorswift_ref9
==> Computing CQT of taylorswift_ref10
==> Computing CQT of taylorswift_ref11
==> Computing CQT of taylorswift_ref12
==> Computing CQT of taylorswift_ref13
==> Computing CQT of taylorswift_ref14
==> Computing CQT of taylorswift_ref15
==> Computing CQT of taylorswift_ref16
==> Computing CQT of taylorswift_ref17
==> Computing CQT of taylorswift_ref18
==> Computing CQT of taylorswift_ref19
==> Computing CQT of taylorswift_ref20
==> Computing CQT of taylorswift_ref21
==> Computing CQT of taylorswift_ref22
==> Computing CQT of taylorswift_ref23
==> Computing CQT of taylorswift_ref24
==> Computing CQT of taylorswift_ref25
==> Computing CQT of taylorswift_ref26
==> Computing CQT of taylorswift_ref27
==> Computing CQT of taylorswift

## Learning PCA Filters

In [5]:
%cd $GIT_DIR
from PCA import *

/home/mjenrungrot/fast.ai/live-song-id


Next, we generate the filters for the hashprint representation by taking PCA on the covariance matrix for all reference songs for this artist. First, we compute the covariance matrix:

In [6]:
num_features = 64
nbins = 121
m = 20 # number of frames

In [7]:
f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'r')
accum_cov = np.zeros((nbins * m, nbins * m))
count = 0

for line in f:
    print('==> Computing covariance matrix for %s'%os.path.basename(line)[:-1])
    Q = np.loadtxt(line[:-1])
    A = getTDE(Q)
    #if A.shape[1] > 1:
    accum_cov += np.cov(A.T)
    count += 1

f.close()

==> Computing covariance matrix for taylorswift_ref1.dat
==> Computing covariance matrix for taylorswift_ref2.dat
==> Computing covariance matrix for taylorswift_ref3.dat
==> Computing covariance matrix for taylorswift_ref4.dat
==> Computing covariance matrix for taylorswift_ref5.dat
==> Computing covariance matrix for taylorswift_ref6.dat
==> Computing covariance matrix for taylorswift_ref7.dat
==> Computing covariance matrix for taylorswift_ref8.dat
==> Computing covariance matrix for taylorswift_ref9.dat
==> Computing covariance matrix for taylorswift_ref10.dat
==> Computing covariance matrix for taylorswift_ref11.dat
==> Computing covariance matrix for taylorswift_ref12.dat
==> Computing covariance matrix for taylorswift_ref13.dat
==> Computing covariance matrix for taylorswift_ref14.dat
==> Computing covariance matrix for taylorswift_ref15.dat
==> Computing covariance matrix for taylorswift_ref16.dat
==> Computing covariance matrix for taylorswift_ref17.dat
==> Computing covarianc

Next, we compute PCA from these covariance matrices.

In [8]:
import scipy.linalg
evals, evecs = scipy.linalg.eig(accum_cov / count)
evecs = evecs[:num_features]
np.savetxt(os.path.join(out_dir, artist + '_model.dat'), evecs)

## Applying Filters to CQT Matrix to generate the Database

In [9]:
evecs = np.loadtxt(os.path.join(out_dir, artist + '_model.dat'))

In order to stay consistency with the keras model, we transpose the filters and the CQT images to have shape ```(width, height)```.

In [10]:
pca_matrix = np.array([vec.reshape((m, -1)) for vec in evecs])
delta = 16
max_pitch_shift = 4
from model import *

/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


For each reference track, we pitch shift the CQT matrix by four pitches, both up and down. For each pitch-shifted version, we compute the hashprint by first passing it through the convolutional network, and then taking the delta feature and thresholding by zero.

The database contains filenames as keys and the pitch-shifted delta features as values.

In [11]:
f = open(os.path.join(out_dir, artist + '_cqtList.txt'), 'r')
db = {}

for line in f:
    print('==> Generating database for %s'%os.path.basename(line)[:-1])
    Q = np.loadtxt(line[:-1]).T
    pitch_shift_Qs = np.empty((2 * max_pitch_shift + 1, ) + Q.shape)
    pitch_shift_Qs[0, :, :] = Q
    for i in range(1, max_pitch_shift + 1):
        pitch_shift_Qs[i, :, :] = pitch_shift_CQT(Q.T, i).T
    for i in range(1, max_pitch_shift + 1):
        pitch_shift_Qs[i + max_pitch_shift, :, :] = pitch_shift_CQT(Q.T, -i).T
    
    conv_1d_net = build_model(pca_matrix, Q.shape, delta=delta)
    
    fpseqs = run_model(conv_1d_net, pitch_shift_Qs)
    delta_fp = fpseqs[:, :, :fpseqs.shape[2] - delta] - fpseqs[:, :, delta:]
    
    db[os.path.basename(line)[:-1]] = np.where(delta_fp > 0, 1, 0)
f.close()
        

==> Generating database for taylorswift_ref1.dat
==> Generating database for taylorswift_ref2.dat
==> Generating database for taylorswift_ref3.dat
==> Generating database for taylorswift_ref4.dat
==> Generating database for taylorswift_ref5.dat
==> Generating database for taylorswift_ref6.dat
==> Generating database for taylorswift_ref7.dat
==> Generating database for taylorswift_ref8.dat
==> Generating database for taylorswift_ref9.dat
==> Generating database for taylorswift_ref10.dat
==> Generating database for taylorswift_ref11.dat
==> Generating database for taylorswift_ref12.dat
==> Generating database for taylorswift_ref13.dat
==> Generating database for taylorswift_ref14.dat
==> Generating database for taylorswift_ref15.dat
==> Generating database for taylorswift_ref16.dat
==> Generating database for taylorswift_ref17.dat
==> Generating database for taylorswift_ref18.dat
==> Generating database for taylorswift_ref19.dat
==> Generating database for taylorswift_ref20.dat
==> Gener

After generating the database, we serialize it and save it to the disk.

In [12]:
import pickle
db_path = os.path.join(out_dir, artist + '_db.pickle')

In [13]:
with open(db_path, 'wb') as handle:
    pickle.dump(db, handle, pickle.HIGHEST_PROTOCOL)

## Matching Query Against Database

First, we load the database from disk:

In [14]:
from search import *
db = {}
with open(db_path, 'rb') as handle:
    db = pickle.load(handle)

We preprocess the query by applying the PCA filters.

In [15]:
query_paths = get_allpaths(artist, list_dir, file_type='query')
def get_query_shape():
    '''
        returns the shape of query file in (width, height)
    '''
    assert len(query_paths) > 0
    cur_file = query_paths[0]
    y, sr = librosa.load(audio_path + cur_file + '.wav')
    Q = librosa.cqt(y, sr=sr, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)
    logQ = preprocess(Q, 3)
    return logQ.T.shape

query_shape = get_query_shape()
querys = np.empty((len(query_paths), ) + query_shape)

for i in range(len(query_paths)):
    cur_file = query_paths[i]
    print('==> Computing CQT of %s'%cur_file)
    y, sr = librosa.load(audio_path + cur_file + '.wav')
    Q = librosa.cqt(y, sr=sr, fmin=130.81, n_bins=121, bins_per_octave=24, hop_length=96)
    logQ = preprocess(Q, 3)
    querys[i, :, :] = logQ.T

conv_1d_net = build_model(pca_matrix, query_shape, delta=delta)
fpseqs = run_model(conv_1d_net, querys)
delta_fp = fpseqs[:, :, :fpseqs.shape[2] - delta] - fpseqs[:, :, delta:]
query_reps = np.where(delta_fp > 0, 1, 0)
    

/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):
/home/mjenrungrot/anaconda3/lib/python3.6/site-packages/librosa/util/utils.py:1725: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(x.dtype, float) or np.issubdtype(x.dtype, complex):


==> Computing CQT of taylorswift_query1
==> Computing CQT of taylorswift_query2
==> Computing CQT of taylorswift_query3
==> Computing CQT of taylorswift_query4
==> Computing CQT of taylorswift_query5
==> Computing CQT of taylorswift_query6
==> Computing CQT of taylorswift_query7
==> Computing CQT of taylorswift_query8
==> Computing CQT of taylorswift_query9
==> Computing CQT of taylorswift_query10
==> Computing CQT of taylorswift_query11
==> Computing CQT of taylorswift_query12
==> Computing CQT of taylorswift_query13
==> Computing CQT of taylorswift_query14
==> Computing CQT of taylorswift_query15
==> Computing CQT of taylorswift_query16
==> Computing CQT of taylorswift_query17
==> Computing CQT of taylorswift_query18
==> Computing CQT of taylorswift_query19
==> Computing CQT of taylorswift_query20
==> Computing CQT of taylorswift_query21
==> Computing CQT of taylorswift_query22
==> Computing CQT of taylorswift_query23
==> Computing CQT of taylorswift_query24
==> Computing CQT of tayl

Then, we run the cross-correlation search algorithm on each of the query representation:

In [16]:
querys = [q for q in query_reps]
refs = list(db.values())
ground_truths = get_querytoref(artist, list_dir)
mrr = calculateMRR(querys, refs, ground_truths)
print('==> MRR for %s is %f'%(artist, mrr))

==> Searching query 0
	-- Time elapsed: 19 seconds
==> Searching query 1
	-- Time elapsed: 18 seconds
==> Searching query 2
	-- Time elapsed: 18 seconds
==> Searching query 3
	-- Time elapsed: 19 seconds
==> Searching query 4
	-- Time elapsed: 18 seconds
==> Searching query 5
	-- Time elapsed: 20 seconds
==> Searching query 6
	-- Time elapsed: 19 seconds
==> Searching query 7
	-- Time elapsed: 18 seconds
==> Searching query 8
	-- Time elapsed: 18 seconds
==> Searching query 9
	-- Time elapsed: 19 seconds
==> Searching query 10
	-- Time elapsed: 18 seconds
==> Searching query 11
	-- Time elapsed: 19 seconds
==> Searching query 12
	-- Time elapsed: 19 seconds
==> Searching query 13
	-- Time elapsed: 18 seconds
==> Searching query 14
	-- Time elapsed: 19 seconds
==> Searching query 15
	-- Time elapsed: 18 seconds
==> Searching query 16
	-- Time elapsed: 18 seconds
==> Searching query 17
	-- Time elapsed: 18 seconds
==> Searching query 18
	-- Time elapsed: 18 seconds
==> Searching query 19